In [1]:
import logging
from pathlib import Path
from Pegasus.api import *
import glob
import os
import pandas as pd
logging.basicConfig(level = logging.DEBUG)

#Properties
props = Properties()
props["dagman.retry"] = "100"
props["pegasus.transfer.arguments"] = "-m 1"
props.write()


# import os
# os.environ['KAGGLE_USERNAME'] = "vedula"
# os.environ['KAGGLE_KEY'] = "482a5c14ced45f63f3698eacb8fa0c62"

# import kaggle
# kaggle.api.dataset_download_files('nikhilpandey360/chest-xray-masks-and-labels/download', path='.', unzip=True)

tc = TransformationCatalog()

unet_wf = Container(
                "unet_wf",
                Container.DOCKER,
                image="docker://vedularaghu/unet_wf:latest"
            )

tc.add_containers(unet_wf)

preprocess = Transformation(
                "preprocess",
                site="condorpool",
                pfn="/usr/bin/preprocess.py",
                is_stageable=False,
                container=unet_wf
            )

data_split = Transformation(
                "data_split",
                site="condorpool",
                pfn="/usr/bin/data_split.py",
                is_stageable=False,
                container=unet_wf
            )

train_model = Transformation( 
                "train_model",
                site="condorpool",
                pfn="/usr/bin/train_model.py",
                is_stageable=False,
                container=unet_wf
            )

predict_masks = Transformation( 
                "predict_masks",
                site="condorpool",
                pfn="/usr/bin/prediction.py",
                is_stageable=False,
                container=unet_wf
            )


tc.add_transformations(preprocess, data_split, train_model).write()

file_list = []
output_list = []

rc = ReplicaCatalog()

for file in glob.glob("./train_images/*.png"):
    f = file.replace("./train_images/", '')
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./train_images/").resolve() / f)
    
for file in glob.glob("./train_masks/*.png"):
    f = file.replace("./train_masks/", '')
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./train_masks/").resolve() / f)
    
for file in glob.glob("./test/*.png"):
    f = file.replace("./test/", '')
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./test/").resolve() / f)    

    
checkpoint_file = "study_checkpoint.pkl"
if not os.path.isfile(checkpoint_file):
    df = pd.DataFrame(list())
    df.to_pickle(checkpoint_file)

rc.add_replica("local", checkpoint_file, Path(".").resolve() / checkpoint_file)
    
rc.write()

for filename in glob.glob("./train_images/*.png"):
    f = filename.replace("./train_images/", '').strip(".png")+"_norm.png"
    output_list.append(File(f))


for filename in glob.glob("./train_masks/*.png"):
    f = filename.replace("./train_masks/", '').strip(".png")+"_norm.png"
    output_list.append(File(f))

for filename in glob.glob("./test/*.png"):
    f = filename.replace("./test/", '').strip(".png")+"_norm.png"
    output_list.append(File(f))

        
wf = Workflow("preprocess")
    
job_preprocess = Job(preprocess)\
                    .add_inputs(*file_list)\
                    .add_outputs(*output_list)

data_split_file = File("data_split.pkl")

job_data_split = Job(data_split)\
                    .add_inputs(*output_list)\
                    .add_outputs(data_split_file)

model = File("model.h5")


job_train = Job(train_model)\
                    .add_checkpoint(File(checkpoint_file), stage_out=True)\
                    .add_inputs(*output_list, data_split_file)\
                    .add_outputs(model)
mask_output = []

for filename in glob.glob("./test/*.png"):
    f = filename.replace("./test/", '').strip(".png")+"_mask.png"
    mask_output.append(File(f))

job_predict = Job(predict_masks)\
                    .add_checkpoint(File(model), stage_out=True)\
                    .add_inputs(*output_list, data_split_file)\
                    .add_outputs(model, *mask_output)


wf.add_jobs(job_preprocess, job_data_split, job_train, job_predict)

try:
    wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)  


################
# pegasus-plan #
################
2020.11.25 12:29:05.898 UTC:
2020.11.25 12:29:05.904 UTC:   -----------------------------------------------------------------------
2020.11.25 12:29:05.909 UTC:   File for submitting this DAG to HTCondor           : preprocess-0.dag.condor.sub
2020.11.25 12:29:05.914 UTC:   Log of DAGMan debugging messages                 : preprocess-0.dag.dagman.out
2020.11.25 12:29:05.921 UTC:   Log of HTCondor library output                     : preprocess-0.dag.lib.out
2020.11.25 12:29:05.927 UTC:   Log of HTCondor library error messages             : preprocess-0.dag.lib.err
2020.11.25 12:29:05.933 UTC:   Log of the life of condor_dagman itself          : preprocess-0.dag.dagman.log
2020.11.25 12:29:05.939 UTC:
2020.11.25 12:29:05.946 UTC:   -no_submit given, not submitting DAG to HTCondor.  You can do this with:
2020.11.25 12:29:05.957 UTC:   -----------------------------------------------------------------------
2020.11.25 12:29:07.078 UTC:  

[##################################################] 100.0% ..Success (Completed: 16, Queued: 0, Running: 0, Failed: 0)



####################
# pegasus-analyzer #
####################
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

Submit Directory   : /home/scitech/shared-data/UNet_wf/scitech/pegasus/preprocess/run0004
Total jobs         :     16 (100.00%)
# jobs succeeded   :     16 (100.00%)
# jobs failed      :      0 (0.00%)
# jobs held        :      0 (0.00%)
# jobs unsubmitted :      0 (0.00%)



######################
# pegasus-statistics #
######################
Your database is compatible with Pegasus version: 5.0.0dev

#
# Pegasus Workflow Management System - http://pegasus.isi.edu
#
# Workflow summary:
#   Summary of the workflow execution. It shows total
#   tasks/jobs/sub workflows run, how many succeeded/failed etc.
#   In case of hierarchical workflow the calculation shows the
#   statistics across all the sub workflows.It shows the following
#   statistics about tasks, jobs and sub workflows.
#